# Lab5-Property extraction (extra assignment)

Copyright, Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

In this notebook, we provide more information about the task of Property Extraction.

Overview of the content covered in this notebook:
1. Introduction to property extraction
2. Building pattern-based extractors
3. Coding pattern-based extractors
4. Processing Wikipedia documents
5. Evaluating extractors

**At the end of this notebook, you will be able to**:
* understand the task of Property Extraction and its relation to similar tasks, like Relation Extraction
* understand the parts of a property extraction tool
* build a pattern-based Property Extractor based on substring matching
* build a pattern-based Property Extractor based on dependency parsing information
* apply an extractor to extract properties from text
* apply extractors on Wikipedia
* evaluate a pattern-based extractor

Throughout this notebook we use the terms attribute, property and relation to mean the same thing a "predicate" in a triple relation.

### Before we start: set up your environment



**1. Install Wikipedia client** In this week's lab session we are going to use Wikipedia. You first need to install a client package to access Wikipedia. From the terminal (with the settings that you use for notebooks) run:

`conda install -c conda-forge wikipedia`

In [1]:
import wikipedia

**2. Internet** Note that you need to have Internet connection to be able to access Wikipedia. If you are not connected or the connection is too slow you get the following error:

`NewConnectionError: <urllib3.connection.HTTPConnection object at 0x1289d3c88>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known`

**3. SpaCy** Another library that we will use in this week's lab sesion is SpaCy and its English model "en_core_web_sm". You should have this already installed because we used this setup in assignment 3. Let's import SpaCy and its English model:

In [2]:
import spacy

model="en_core_web_sm"

nlp = spacy.load(model)
print("Info: Loaded model '%s'" % model)

Info: Loaded model 'en_core_web_sm'


**4. NLTK** NLTK is a standard language processing module in Python. This should be installed by defult with conda, but in case this is not the case, you can install it with:

`conda install -c anaconda nltk`

In [3]:
from nltk import Tree

The Tree module is used in the lab5_utils.py file, where auxiliary classes are given to extract properties from text. We discuss these in more detail below. The above import is also done in lab5_utils.

If the above code blocks did not yield any error, then you are all set up for this week's session. Let's start ;)

### 1. Introduction to property extraction

In the task of entity linking (see Lab4), we performed disambiguation of entity mentions in text by making a connection to the correct referrent for a mention in a knowledge base. Although these knowledge bases are typically fairly large, they are far from complete. Tasks like property extraction and relation extraction help to make knowledge bases more complete, by extracting knowledge from text.

The task of property extraction aims to fill knowledge bases with information about properties of entities that we find in text. There are other tasks that are similar to it, such as: 
* slot filling, where we attempt to complete entity information according to some schema
* relation extraction - given two entities, what is their relation (for example, in Microsoft X Bill_Gates, the relation X is `hasCEO`)
* knowledge base completion, where we usually complete a knowledge base by inference from existing structured information (not from text).
* open information extraction - no schema available, very little restrictions on what to extract, disambiguation is non-trivial

In all these tasks, including property extraction, we typically extract "pieces of knowledge" in the form of **a triple**. A triple consists of three elements: a subject, a predicate, and an object (not to be confused with grammatical functions in a syntactic dependency structure). An example of a triple is:

```[Barack_Obama hasAge 57]```

Here, Barack_Obama is a subject, hasAge is a predicate/relation, and 57 is an object. In the Semantic Web, the subjects and the predicates of a triple are always URIs; the objects can be either a URI (like Barack_Obama), or a literal (like 57, or "Barack").

Hence, property extraction typically requires us to:
1. **detect** a property value in text (e.g., 57) and an entity it belongs to ("Barack")
2. **interpret** both the property value (57 as a number) and the entity ("Barack" means Barack_Obama)
3. **find their relation** - the connection between Barack_Obama and 57 is the relation hasAge

In this sense, the task of property extraction builds on top of the output of NERC and NERD.

**Challenges** The property extraction task is difficult for reasons similar to those we have discussed with entity linking: ambiguity (of entities, relations, and property values), variation (of entities, values, and relations), and vagueness (when insufficient details or information is given). Futhermore,  relations can sometimes span multiple sentences or require a lot of world knowledge in order to understand them.

### 2. Building extractors

The main focus of this week's lab session is on creating our own property extractors. 

#### 2.1 Methods
Building automatic property extractors is not trivial, because it requires multiple steps, and these steps in practice might differ a lot across different properties.

There are two common methods for extracting properties from text: pattern matching and distant supervision (supervised machine learning). 
* The most basic approach for extracting properties from text is by pattern matching. This approach is transparent, but it requires us to define the patterns for each of the properties separately. For example, for the property "birthplace", we can use the pattern: "X, born in Y", or "X from Y". Typically, the patterns are combined with information on entity types to help their precision. For example, we will check whether indeed X is a person and Y is indeed a location in the above example. We can also filter syntactic depedencies, e.g. X should be the syntactic subject of the predicate "born in".
* The second approach, distant supervision, relies on knowledge base information that is loosely based on a text. For example, you can think of a Wikipedia document that describes Donald Trump on the one hand, and structured information from Wikidata that tells us that he is born in 1946. However, we don't know if this information is explicitly mentioned in the Wikipedia text and if so, where and how. With the distant supervision method, we train for example a recurrent neural network on top of this kind of output, and hope that the neural network will learn the patterns in which this properperty is typically given in text.

We will use a pattern matching approach to build our extractors in this week's lab session. If you are curious about how to build a distant supervision extractor, you can check Snorkel (their [introductory notebooks](https://github.com/HazyResearch/snorkel/tree/master) are quite user-friendly).

#### 2.2 Building a pattern-based extractor

Typically, a pattern-based extractor consists of several parts:
1. find a mention of a specific property (for example, money or birth date) in text
2. assign this mention to some subject entity
3. normalize the property value
4. normalize the subject entity

**Example** Let's say we want to extract values for `founding year` in the following paragraph from Wikipedia:

"Juventus F.C. is an Italian professional football club based in Turin. The club was founded in March 1897 by a group of Torinese students."

**Step I** First, we need to find property values that contain information about founding years. For example, we can use the pattern "founded in" to extract the property value `March 1897` in the second sentence.

**Step II** Next, we need to see to which subject this property belongs. Assuming that we perform dependency parsing of the sentences, we can find that the relation "founded in" has a syntactic subject `The club`. At this point, we can extract the following relation:

The club FOUNDING_YEAR March 1897

Syntactially this is the correct way to extract the relation. However, the relation is not really very useful yet - we need to normalize its subject and object somehow to make it useful in a semantic sense.

**Step III** Hence, we can normalize the value "March 1897" to a year value `1897`, for example, by looking for 4-digit numbers in the phrase.

**Step IV** Then, we can normalize "The club" to `Juventus F.C.` by using entity coreference between the two sentences. We can then disambiguate the mention to https://en.wikipedia.org/wiki/Juventus_F.C., or `Juventus_F.C.` for brevity. This finally leads us to the following relation:

`Juventus_F.C. FOUNDING_YEAR 1897`

which looks much more useful (and it is on a semantic level, so we can store it in a knowledge base if we would like to).

### 3. Coding pattern-based extractors

**Implementing the four steps from 2.2** We will start by searching entity mentions of some type (e.g., nationality or date) by using SpaCy's named entity recognizer. 

Often this is not enough for step I, because an entity of a certain type can be a value of different properties. For example, the date "1946" can be a year of birth, a year of death, a founding year of a company, a year of starting/ending professional activity, etc. We will hence refer to this property value generation step as step Ia.

In step Ib then, we can either:
- check whether we find some keywords before the phrase (such as "born in" or "founded in"). We will build such an extractor in 3.1. 
- or use the dependency tree (also from SpaCy) to find the word that is associated with this value with a given dependency relation. We will do this in 3.2.

For step II of assigning the value to some entity phrase, we will look for the closest entity to this property value and assign it to that one. Step III would mostly differ per kind of property value - e.g., it is different to normalize dates than it is to normalize company names. We will not deal with step IV in this week's session.

**Helper functions** We will use a number of helper functions in 3.1 and 3.2. These functions are located in the file `lab5_utils.py`. You don't need to know their functionality in detail (if you are curious, feel free to check their code). Let's load these helper functions first.

In [4]:
import lab5_utils as utils

### 3.1 Using substring matching
We will use substring matching to look for founding years of organizations. 
We will use three simple patterns for this purpose: 'founded in X', 'established in X', 'created in X'. 
As mentioned before, we will also make sure that X is an entity of type DATE.

You can find the full set of entity types recognized by SpaCy [here](https://spacy.io/usage/linguistic-features).

We will use some helper functions from the file `lab5_utils.py` in this example:

* `get_entities_of_type` to find all entities with a specific type - we will use this to find the property values in step Ia.
* `pattern_found_on_the_left` tells us whether one of the patterns appears immediately on the left of the property value. This function internally calls the function `check_for_pattern` for each of the patterns. This is step Ib.
* `find_closest_entity` to find the closest entity of type X to a property value (step II)
* `extract_year_from_date` - given a full date, this function extracts a four-digit year number (step III).


Let's now write the main function to extract date relations based on patterns. You can find the steps covered in the comments.

In [5]:
def extract_date_relations(doc, patterns):
    """
    Extract date properties from a spaCy document and assign them to an entity.
    """
    
    property_value_type='DATE'
    target_entity_type='ORG'
    
    # the following 3 lines merge entities and noun chunks into one token
    # The tokens "New" "York" will be retokenized into a single new token "New York"
    # Check out the spaCy documentation for details
    # The tokens in doc will be modified after calling this function
    # this is useful in our cases, so we will always do it.
    spans = list(doc.ents) + list(doc.noun_chunks)
    for span in spans:
        span.merge() ### will be replaced by the function "retokenize"
    
    relations = {}
    
    # step Ia - generate possible property values
    dates=utils.get_entities_of_type(property_value_type, doc)
    
    for date in dates:
        # step Ib - is one of our patterns found before the date 
        if utils.pattern_found_on_the_left(doc, date.i, patterns):
            # step II - find the closest entity of some target type
            org=utils.find_closest_entity(doc.ents, date.idx, target_entity_type)
            # step III - normalize the year
            year=utils.extract_year_from_date(date.text)
            if year and org:
                relations[org]=year

    return relations        

We can test whether the founding date extraction works as we expect it to.

In [6]:
founded_patterns=['established in', 'founded in', 'created in']

text='Airbus was founded in December 1991, not in January 1992.'

doc = nlp(text)
date_relations=extract_date_relations(doc, founded_patterns)
print(date_relations)

{'Airbus': 1991}


#### 3.2 Using syntax dependencies


Here, we extract organizations (entities labelled as `ORG`) and then check the dependency tree to find whether its syntax dependencies with other keywords (such as "developed") are as expected.

In order to know which dependencies we need, let's first print the dependency tree of one example sentence and see which dependencies do we want to look for.

In [7]:
text = "The 88000 is an architecture developed by Motorola in the 1980s."
example_doc = nlp(u'' + text) 

for sent in example_doc.sents:
    utils.to_nltk_tree(sent.root).pretty_print()

                         is_VBZ_ROOT                                                
     _________________________|_____________                                         
    |           |                    architecture_NN_                               
    |           |                          attr                                     
    |           |              _____________|________________                        
    |           |             |                       developed_VBN_ac              
    |           |             |                              l                      
    |           |             |              ________________|_______________        
    |           |             |             |                            in_IN_prep 
    |           |             |             |                                |       
    |     88000_CD_nsubj      |        by_IN_agent                     1980s_CD_pobj
    |           |             |             |                

For each of the nodes of the tree, we see three pieces of information: the word itself ("Motorola"), the part-of-speech tag ("NNP"), and the dependency tag ("pobj"). You can find an explanation of the dependency labels [here](https://github.com/clir/clearnlp-guidelines/blob/master/md/specifications/dependency_labels.md). 

In the example, we want to find cases where Motorola has a specific dependency relation with the word "developed". From the tree, you can see that we are looking for cases where our property value Motorola is a prepositional object (pobj) and the keyword we need ("developed") is two levels above Motorola (this is the head of the head of Motorola). We also see that this keyword has a dependency label "acl" (clausal modifier of a noun). 

Knowing this, we can now write a dependency function that searches for this kind of dependency patterns.

In [8]:
def fitting_dependency(token, predicates):
    """
    Check whether the we find the right keyword in the correct part of the dependency tree.
    """
    # Find prepositional objects that have a head with dependency label 'agent'
    # and its head has a dependency label 'acl'
    # Also, we make sure that the head of the head of our object is one of our keywords.
    if token.dep_ == 'pobj' and token.head.dep_ == 'agent' and token.head.head.dep_ =='acl':
        pred=token.head.head
        if pred.text in predicates:
            return True
        else:
            return False
    else:
        return False

Now we can write our main function to extract device manufacturers from text. The helper functions we will use are:
    
- `get_entities_of_type` to find all entities with a specific type - we will use this to find the property values in step Ia.
- `fitting_dependency` - to find whether any wanted keyword appears in the correct dependency relation to our property value (step Ib).
- `find_closest_entity` to find the closest entity of type X to a property value (step II)

In [17]:
def extract_manufacturer(doc, predicates, main_entity):
    
    property_value_type='ORG'
    target_entity_type='PRODUCT'
    
    # the following 3 lines merge entities and noun chunks into one token
    # this is useful in our cases, so we will always do it.
    spans = list(doc.ents) + list(doc.noun_chunks)
    for span in spans:
        span.merge()
    
    relations={}
    
    # step Ia - generate possible property values
    manus=utils.get_entities_of_type(property_value_type, doc)
    
    for manu in manus:
        # step Ib - do we find the right keyword in the correct part of the dependency tree?
        if fitting_dependency(manu, predicates):
            # step II - find the closest entity of some target type
            device=utils.find_closest_entity(doc.ents, manu.idx, target_entity_type)
            # Devices are often not recognized properly by SpaCy - 
            # if we find no device, we assume that the relation is about the main entity of the document
            if not device:
                device=main_entity
            if device and manu:
                relations[device]=manu.text
    return relations

We can test whether the manufacturer extraction works as we expect it to.

In [18]:
manu_predicates=['manufactured', 'produced', 'developed']
text='The 88000 (m88k for short) is a RISC instruction set architecture (ISA) developed by Motorola during the 1980s.'
main_entity='Motorola 88000'

doc = nlp(text)
manu_relations=extract_manufacturer(doc, manu_predicates, main_entity)
print(manu_relations)

{'Motorola 88000': 'Motorola'}


In [15]:
manu_predicates=['manufactured', 'produced', 'developed']

good_text = 'Apple developed the 88000'
bad_text = 'Apple is an American multinational technology company that designs, develops, and sells Apple TVs, Iphones and Ipads.'
main_entity='Apple'
doc = nlp(good_text)
manu_relations=extract_manufacturer(doc, manu_predicates, main_entity)
print(manu_relations)

{}


#### 3.3 Running all our extractors

Now we will write a function that runs all our extractors on a bunch of text documents paired with entities. The input is a dictionary consisting of the entity name as the key and the text sentence to find any of the relations as the value.

In [36]:
def get_all_relations(texts):
    """
    Run all our relations on a number of text documents.
    """
    founding_years={}
    manufacturers={}

    founded_patterns=['established in', 'founded in', 'created in']
    manu_predicates=['manufactured', 'produced', 'developed']

    for main_entity, text in texts.items():
        doc = nlp(text)
        date_relations=extract_date_relations(doc, founded_patterns)
        manu_relations=extract_manufacturer(doc, manu_predicates, main_entity)

        founding_years.update(date_relations)
        manufacturers.update(manu_relations)

    return founding_years, manufacturers

In [37]:
texts = {
    'Motorola 88000': 'The 88000 (m88k for short) is a RISC instruction set architecture (ISA) developed by Motorola during the 1980s.',
    'Airbus': 'Airbus was founded in December 1991, not in January 1992.',
}

print("Info: Processing %d texts" % len(texts.keys()))
print()
print('**Extracted relations:**')
print()

founding_years, manufacturers=get_all_relations(texts)
print('Founding years')
print(founding_years)
print('Manufacturers')
print(manufacturers)
    
# Expected output:
# {'Airbus': 1991}
# {'Motorola 88000': Motorola}

Info: Processing 2 texts

**Extracted relations:**

Founding years
{'Airbus': 1991}
Manufacturers
{'Motorola 88000': 'Motorola'}


### 4. Processing wikipedia

Now that we know how to run our extractors on some text documents, we can do this on a larger scale. As an illustration, here we will load the first 3 sentences from a few Wikipedia documents and try to extract properties from them.

In [63]:
entities=["Motorola 88000", "Nokia", "The Coca-Cola Company"]

texts={}

for entity in entities:
    print(entity)
    wp = wikipedia.page(entity)
    # get the first 3 sentences of a wikipedia article
    first_three_sentences=wp.content.split('.')[:3]
    entity_text=('').join(first_three_sentences)
    # create a dictionary (JSON) where the key is your entity, and the value is its 3-sentences wikipedia text. 
    texts[entity]=entity_text
    print(entity_text)
    print()
    
system_founding_years, system_manufactures = get_all_relations(texts)
print(system_founding_years, system_manufactures)

Pacino
Alfredo James Pacino (; Italian: [paˈtʃiːno]; born April 25, 1940) is an American actor and filmmaker In a career spanning over five decades, he has received many awards and nominations, including an Academy Award, two Tony Awards and two Primetime Emmy Awards He is one of the few performers to have received the Triple Crown of Acting



NameError: name 'get_all_relations' is not defined

### 5. Evaluating extractors

We will evaluate the extractors by computing a precision, recall, and F1-score per document. We will only check the extracted values for the main entity in the document, not for any of the others.

Similar as with entity linking, we will decide on true positives, false positives, false negatives per textual unit and not per class. In this case, the textual unit we will use is the document. 

In [13]:
properties=['founding_year', 'nationality']
gold={}
gold_manufacturers={'Motorola 88000': 'Motorola'}
gold_founding_years={'Nokia': 1865, 'The Coca-Cola Company': 1886}

print('***Evaluate founding years***')
precision, recall, f1=utils.evaluate_property(system_founding_years, gold_founding_years)
print("precision: %f, \nrecall: %f, \nF1-score: %f\n" % (precision, recall, f1))

print('***Evaluate manufacturers***')
precision, recall, f1=utils.evaluate_property(system_manufactures, gold_manufacturers)
print("precision: %f, \nrecall: %f, \nF1-score: %f" % (precision, recall, f1))

***Evaluate founding years***
precision: 0.000000, 
recall: 0.000000, 
F1-score: 0.000000

***Evaluate manufacturers***
precision: 1.000000, 
recall: 1.000000, 
F1-score: 1.000000


## End of this notebook